<a href="https://colab.research.google.com/github/SunilCotterill/steph-curry-nba-data-analysis/blob/main/steph-curry-popcorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nba Analysis**
<br>



This notebook seeks to find out if there was any relation between Steph Curry's performance, during his unanimous MVP season, and the quality of popcorn served in the arena he was playing, based on his rating.

In [11]:
pip install nba_api

In [12]:
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.static import teams
import json
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playergamelogs
import matplotlib.pyplot as plt
import seaborn as sns

## Getting Data Using the NBA_Api
<br>

Getting Steph's id

In [13]:
steph = players.find_players_by_full_name('Stephen Curry')

In [14]:
steph_id=steph.pop().get('id')

Getting a list of all the NBA teams

In [15]:
nba_teams = teams.get_teams()
nba_teams
df_teams = pd.json_normalize(nba_teams)

#POPCORN and his games

In [16]:
popcorn_ranking=pd.read_excel('/content/drive/MyDrive/DataFest/CurryPopcornRankings.xlsx')

Steph's popcorn data taken from: https://data.world/makeovermonday/2019w17

In [17]:
#gamefinder = leaguegamefinder.LeagueGameFinder(player_id_nullable=steph_id)
#games = gamefinder.get_data_frames()[0]
#games.head

In [18]:
#checking if he played all games from https://stackoverflow.com/questions/54426845/how-to-check-if-a-pandas-dataframe-contains-only-numeric-column-wise
#pd.to_numeric(games['MIN'], errors='coerce').notnull().all

Getting all the games for Steph's MVP season

In [19]:
games_logs = playergamelogs.PlayerGameLogs(player_id_nullable=steph_id, season_nullable='2015-16' )

ReadTimeout: ignored

In [ ]:
real_logs=games_logs.get_data_frames()[0]
real_logs.head

In [ ]:
#checking data to ensure he played all the games
#from: https://stackoverflow.com/questions/54426845/how-to-check-if-a-pandas-dataframe-contains-only-numeric-column-wise
print(pd.to_numeric(real_logs['MIN'], errors='coerce').notnull().all)

print(real_logs['MIN'].nsmallest(3))
print(real_logs.columns.values)
real_logs['MATCHUP'].head

In [ ]:
#Getting the teams he played and stats for each game in on df
test_df = real_logs[['MATCHUP', 'PLUS_MINUS','NBA_FANTASY_PTS','PTS', 'TOV', 'PF']]
test_df['MATCHUP'].str[-3:]
test_df['MATCHUP'] = test_df['MATCHUP'].str[-3:]
test_df.head

In [ ]:
teams_abrev = df_teams[['abbreviation','full_name']]
#from https://stackoverflow.com/questions/17426292/what-is-the-most-efficient-way-to-create-a-dictionary-of-two-pandas-dataframe-co
city_abrev=pd.Series(teams_abrev.full_name.values,index=teams_abrev.abbreviation).to_dict()
merged_df=test_df
#adding the opposing team name based on the city for each row
merged_df['CITY'] =merged_df['MATCHUP'].map(city_abrev)
print(merged_df.head)


In [ ]:
print(popcorn_ranking.head)
#spaces are scary

#woo is the df 
woo = popcorn_ranking.rename(columns={"TOTAL SCORE": "TOTAL_SCORE"}, errors="raise")
woo.columns.values


In [ ]:
#combining all the popcorn ratings and games together
  #yes this is the best way to do it, copying and pasting code is always the best practice


popcorn_city=pd.Series(woo.TOTAL_SCORE.values,index=woo.TEAM).to_dict()
popcorn_city['Los Angeles Clippers']=popcorn_city['Los Angeles Clippers/Lakers']
popcorn_city['Los Angeles Lakes']=popcorn_city['Los Angeles Clippers/Lakers']

popcorn_city_FRESHNESS=pd.Series(woo.FRESHNESS.values,index=woo.TEAM).to_dict()
popcorn_city_FRESHNESS['Los Angeles Clippers']=popcorn_city_FRESHNESS['Los Angeles Clippers/Lakers']
popcorn_city_FRESHNESS['Los Angeles Lakes']=popcorn_city_FRESHNESS['Los Angeles Clippers/Lakers']

popcorn_city_SALTINESS=pd.Series(woo.SALTINESS.values,index=woo.TEAM).to_dict()
popcorn_city_SALTINESS['Los Angeles Clippers']=popcorn_city_SALTINESS['Los Angeles Clippers/Lakers']
popcorn_city_SALTINESS['Los Angeles Lakes']=popcorn_city_SALTINESS['Los Angeles Clippers/Lakers']

popcorn_city_CRUNCHINESS=pd.Series(woo.CRUNCHINESS.values,index=woo.TEAM).to_dict()
popcorn_city_CRUNCHINESS['Los Angeles Clippers']=popcorn_city_CRUNCHINESS['Los Angeles Clippers/Lakers']
popcorn_city_CRUNCHINESS['Los Angeles Lakes']=popcorn_city_CRUNCHINESS['Los Angeles Clippers/Lakers']

popcorn_city_BUTTER=pd.Series(woo.BUTTER.values,index=woo.TEAM).to_dict()
popcorn_city_BUTTER['Los Angeles Clippers']=popcorn_city_BUTTER['Los Angeles Clippers/Lakers']
popcorn_city_BUTTER['Los Angeles Lakes']=popcorn_city_BUTTER['Los Angeles Clippers/Lakers']

popcorn_city_PRESENTATION=pd.Series(woo.PRESENTATION.values,index=woo.TEAM).to_dict()
popcorn_city_PRESENTATION['Los Angeles Clippers']=popcorn_city_PRESENTATION['Los Angeles Clippers/Lakers']
popcorn_city_PRESENTATION['Los Angeles Lakes']=popcorn_city_PRESENTATION['Los Angeles Clippers/Lakers']


merged_df['POPCORN_RANKING'] =merged_df['CITY'].map(popcorn_city)
merged_df['POPCORN_FRESHNESS'] =merged_df['CITY'].map(popcorn_city_FRESHNESS)
merged_df['POPCORN_SALTINESS'] =merged_df['CITY'].map(popcorn_city_SALTINESS)
merged_df['POPCORN_CRUNCHINESS'] =merged_df['CITY'].map(popcorn_city_CRUNCHINESS)
merged_df['POPCORN_BUTTER'] =merged_df['CITY'].map(popcorn_city_BUTTER)
merged_df['POPCORN_PRESENTATION'] =merged_df['CITY'].map(popcorn_city_PRESENTATION)
print(merged_df.head)

#Displaying data

In [ ]:
#DATA TIME
merged_df.columns.values

In [ ]:
#thanks jack
sns.set(style="ticks")
sns.pairplot(merged_df, x_vars=['POPCORN_RANKING','POPCORN_FRESHNESS','POPCORN_SALTINESS','POPCORN_CRUNCHINESS','POPCORN_BUTTER', 'POPCORN_PRESENTATION'], y_vars=['PLUS_MINUS','NBA_FANTASY_PTS','PTS','TOV','PF'], palette = 'pastel', hue="CITY", height=2.5)
plt.show()

"From the above graphs it doesn't look like there's a strong correlation" - Sunil's analysis. 
The end!